# Scrapping Website P3DN

#### Import Bahan Penting

In [1]:
import asyncio
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import nest_asyncio
import os
import json
from datetime import datetime

#### Check Web Driver

In [23]:
# Inisialisasi driver Chrome
driver = webdriver.Chrome()
# Buka halaman web
driver.get('https://tkdn.kemenperin.go.id/rekap.php')
# Maksimalkan ukuran jendela browser
driver.maximize_window()

#### Ujicoba Scraping Data

In [30]:
#//*[@id="content"]/div/div/div/div[1]

kelompok_barang_jamak=driver.find_elements("xpath",'//*[@id="content"]/div/div/div/div[1]/div[2]/table/tbody/tr/td[2]')

i = 1

for kelompok_barang in kelompok_barang_jamak:
    nama = kelompok_barang.text.strip()
    if nama:
        print(f"Kelompok Barang {i}: {nama}")
        i += 1


Kelompok Barang 1: AC-powered adjustable hospital bed
Kelompok Barang 2: Acetic Acid Glacial
Kelompok Barang 3: Acetone
Kelompok Barang 4: Acetone
Kelompok Barang 5: Acetonitrile LC MS
Kelompok Barang 6: ACICLOVIR
Kelompok Barang 7: AMLODIPINE BESILATE
Kelompok Barang 8: Ammonium Acetate
Kelompok Barang 9: Ammonium Chloride
Kelompok Barang 10: Aquadest
Kelompok Barang 11: Baju Kebaya
Kelompok Barang 12: Baju Kurung
Kelompok Barang 13: Basreng Extreme
Kelompok Barang 14: Belajar Sambil Bermain Interaktif Paud
Kelompok Barang 15: Cardiac Monitor (including cardiotachometer and rate alarm)
Kelompok Barang 16: Cardiac Monitor (including cardiotachometer and rate alarm)
Kelompok Barang 17: Chloral Hydrate
Kelompok Barang 18: Chlorobenzene
Kelompok Barang 19: CLAMP HOOK
Kelompok Barang 20: Copper Sulphate Pentahydrate
Kelompok Barang 21: Corina
Kelompok Barang 22: Cross Over
Kelompok Barang 23: Cyclohexane
Kelompok Barang 24: Dichloromethane
Kelompok Barang 25: DICLOFENAC SODIUM
Kelompok Bar

#### Ujicoba Buka Link

In [ ]:
def buka_kelompok_barang(kd):
    """
    Membuka URL kelompok barang berdasarkan input kd.

    Args:
        kd (int): Kode kelompok barang (1-8).
    """

    # Inisialisasi WebDriver (pastikan Anda memiliki driver yang sesuai di PATH)
    driver = webdriver.Chrome()  # Ganti Chrome() dengan browser yang Anda gunakan (Firefox(), Edge(), dll.)

    try:
        # Membuat URL dinamis
        url = f"https://tkdn.kemenperin.go.id/sertifikat_idx.php?kd={kd}"

        # Membuka URL
        driver.get(url)
        print(f"Berhasil membuka URL: {url}")

    except Exception as e:
        print(f"Terjadi kesalahan: {e}")

# Contoh penggunaan fungsi
kd_input = int(input("Masukkan kode kelompok barang (1-8): "))
if 1 <= kd_input <= 8:
    buka_kelompok_barang(kd_input)
else:
    print("Kode kelompok barang tidak valid. Masukkan angka antara 1 dan 8.")

Berhasil membuka URL: https://tkdn.kemenperin.go.id/sertifikat_idx.php?kd=4


#### Ujicoba Melakukan Scrapping Data P3DN

In [ ]:
nest_asyncio.apply()  # Terapkan nest_asyncio

async def buka_kelompok_barang(kd):
    """Membuka URL kelompok barang berdasarkan input kd."""
    driver = webdriver.Chrome()  # Ganti Chrome() dengan browser Anda jika perlu
    try:
        url = f"https://tkdn.kemenperin.go.id/sertifikat_idx.php?kd={kd}"
        driver.get(url)
        print(f"Berhasil membuka URL: {url}")
        return driver  # Mengembalikan objek driver
    except Exception as e:
        print(f"Terjadi kesalahan: {e}")
        return None  # Mengembalikan None jika terjadi kesalahan

async def scrap_data_kelompok_barang_pagination(driver):
    """Melakukan scrapping data dari tabel kelompok barang dengan pagination."""
    if driver is None:
        return  # Keluar jika driver None (gagal membuka URL)

    try:
        # Menunggu tabel hasil dimuat
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div/div/div/div[1]/div[2]/table/tbody'))
        )

        # Menampilkan header tabel
        header = ["No.", "Perusahaan", "Jenis Produk", "Spesifikasi", "Tipe", "Merk", "Nilai TKDN"]
        print("\t".join(header))

        page_number = 1
        while True:
            # print(f"Scrapping data dari halaman {page_number}")

            # Mendapatkan baris dari tabel
            rows = driver.find_elements(By.XPATH, '//*[@id="content"]/div/div/div/div[1]/div[2]/table/tbody/tr')

            # Menampilkan data tabel
            for i, row in enumerate(rows, start=(page_number - 1) * len(rows) + 1):
                cells = row.find_elements(By.XPATH, './td[position()>=1 and position()<=7]')
                row_data = [str(i)] + [cell.text.strip() for cell in cells]  # Tambahkan nomor urut
                print("\t".join(row_data))

            # Mencari tombol halaman berikutnya
            next_page = driver.find_elements(By.XPATH, '//*[@id="content"]/div/div/div/div/div[1]/div/div[2]/ul/li[position()=last()]/a')

            if next_page and "..." not in next_page[0].text:
                next_page[0].click()
                page_number += 1
                time.sleep(2)  # Tambahkan penundaan untuk menunggu halaman dimuat
                # Menunggu tabel hasil dimuat setelah klik halaman berikutnya
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div/div/div/div[1]/div[2]/table/tbody'))
                )
            else:
                break  # Tidak ada halaman berikutnya atau halaman terakhir, keluar dari loop

    except Exception as e:
        print(f"Terjadi kesalahan saat scrapping data: {e}")
    finally:
        driver.quit()

async def main():
    """Fungsi utama untuk menjalankan skrip asinkron."""
    kd_input = int(input("Masukkan kode kelompok barang (1-8): "))
    if 1 <= kd_input <= 8:
        driver = await buka_kelompok_barang(kd_input)
        if driver:  # Pastikan driver tidak None sebelum melanjutkan
            await scrap_data_kelompok_barang_pagination(driver)
    else:
        print("Kode kelompok barang tidak valid. Masukkan angka antara 1 dan 8.")

if __name__ == "__main__":
    asyncio.run(main())

Berhasil membuka URL: https://tkdn.kemenperin.go.id/sertifikat_idx.php?kd=4
No.	Perusahaan	Jenis Produk	Spesifikasi	Tipe	Merk	Nilai TKDN
1
2	1.	PT Mulia Jaya Mandiri Jakarta	Cross Over	2-3/8 s/d 24 inch, Material L-80	Connection adapter	-	27.28%
3	2.	PT Molden Patra Sejahtera	Pup Joint	2-3/8 inch s/d 4-1/2 inch	Carbon Steel	-	34.23%
4	3.	PT Mulia Jaya Mandiri Jakarta	X-Mass Tree Assy	2.1/16 inch s/d 7.1/16 inch, Tekanan 2000 PSI s/d 15000 PSI, PSL Level 1-3G, Material AA,BB,DD,EE, Temperature U, X, Y, (Bahan Baku Lokal).	X-Mass Tree Assy	-	34.20%
5	4.	PT Mulia Jaya Mandiri Jakarta	Coupling	2-3/8 s/d 20 inch, Material L-80, CR	Connection adapter	-	18.00%
6	5.	PT Mulia Jaya Mandiri Jakarta	Coupling	2-3/8 s/d 20 inch, Material L-80	Connection adapter	-	28.18%
7	6.	PT Mulia Jaya Mandiri Jakarta	Cross Over	2-3/8 s/d 13-3/8 inch, Material L-80, CR	Connection adapter	-	17.94%
8	7.	PT DAESHIN FLANGE FITTING INDUSTRI	FITTING	Size 1/2 inch - 4 inch	Hex Bushing	-	25.38%
9	8.	PT VETCO GRAY INDONES

#### Ujicoba Scrapping Data P3DN dan save dalam format JSON

In [2]:
nest_asyncio.apply()

async def buka_kelompok_barang(kd):
    """Membuka URL kelompok barang berdasarkan input kd."""
    driver = webdriver.Chrome()  # Ganti Chrome() dengan browser Anda jika perlu
    try:
        url = f"https://tkdn.kemenperin.go.id/sertifikat_idx.php?kd={kd}"
        driver.get(url)
        print(f"Berhasil membuka URL: {url}")
        return driver
    except Exception as e:
        print(f"Terjadi kesalahan: {e}")
        return None

async def scrap_data_kelompok_barang_pagination(driver, kd):
    """Melakukan scrapping data dari tabel kelompok barang dengan pagination."""
    if driver is None:
        return

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div/div/div/div[1]/div[2]/table/tbody'))
        )

        header = ["perusahaan", "jenis_produk", "spesifikasi", "tipe", "merk", "nilai_tkdn"]
        print("\t".join(header))

        page_number = 1
        all_data = []

        while True:
            rows = driver.find_elements(By.XPATH, '//*[@id="content"]/div/div/div/div[1]/div[2]/table/tbody/tr')

            for row in rows:
                cells = row.find_elements(By.XPATH, './td[position()>=2 and position()<=7]')  # Mulai dari kolom kedua (Perusahaan)
                row_data = [cell.text.strip() for cell in cells]
                print("\t".join(row_data))
                all_data.append(dict(zip(header, row_data)))

            next_page = driver.find_elements(By.XPATH, '//*[@id="content"]/div/div/div/div/div[1]/div/div[2]/ul/li[position()=last()]/a')

            if next_page and "..." not in next_page[0].text:
                next_page[0].click()
                page_number += 1
                time.sleep(2)
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div/div/div/div[1]/div[2]/table/tbody'))
                )
            else:
                break
        return all_data

    except Exception as e:
        print(f"Terjadi kesalahan saat scrapping data: {e}")
    finally:
        driver.quit()

def create_folder(kd):
    """Membuat folder hasil scrapping."""
    folder_name = f"hasil-scrapping-kd-{kd}"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    return folder_name

def save_data_to_json(data, kd):
    """Menyimpan data ke file JSON."""
    if not data:
        return
    
    folder_name = create_folder(kd)
    now = datetime.now()
    timestamp = now.strftime("%Y-%m-%d--%H-%M-%S")
    file_name = f"hasil-scrapping-kd-{kd}-{timestamp}.json"
    file_path = os.path.join(folder_name, file_name)

    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"Data disimpan di: {file_path}")

async def main():
    """Fungsi utama untuk menjalankan skrip asinkron."""
    kd_input = int(input("Masukkan kode kelompok barang (1-8): "))
    if 1 <= kd_input <= 8:
        driver = await buka_kelompok_barang(kd_input)
        if driver:
            all_data = await scrap_data_kelompok_barang_pagination(driver, kd_input)
            save_data_to_json(all_data, kd_input)
    else:
        print("Kode kelompok barang tidak valid. Masukkan angka antara 1 dan 8.")

if __name__ == "__main__":
    asyncio.run(main())

Berhasil membuka URL: https://tkdn.kemenperin.go.id/sertifikat_idx.php?kd=8
perusahaan	jenis_produk	spesifikasi	tipe	merk	nilai_tkdn

PT SARANA BANGUN PUSAKA	Pagar Pembatas/Pengaman Jalan (Guard Rail)	Beam : 4320mm, tebal 2,7mm ; Post: 1800mm, tebal : 6mm ; Block Post 350mm dan 400mm, tebal : 4,5mm	Tipe A	-	56.79%
PT SARANA BANGUN PUSAKA	Pagar Pembatas/Pengaman Jalan (Guard Rail)	Beam : 4320mm, tebal 2,7mm ; Post: 1800mm, tebal : 4,5mm ; Block Post 350mm dan 400mm, tebal : 4,5mm	Tipe B	-	53.42%
PT Metalindo Teratai Putra	Tabung Baja LPG	Operating Pressure 18,6 Kg/cm2	5,5 Kg	MTP	70.23%
PT ARTAS ENERGI PETROGAS	Carbon Steel Seamless Casing, Non-Heat Treatment	API 5CT, As Rolled / Green Pipe, Grade H40, J55, K55, Diameter 4 1/2 inch – 13 3/8 inch, R1, R2, R3, PE	Casing Plain End	1ST	42.28%
PT ARTAS ENERGI PETROGAS	Carbon Steel Seamless Line Pipe, Heat Treatment	API 5L, Gr. BQ, X42N/Q, X46N/Q, X52N/Q, X56N/Q, X60N/Q/QO/QP, X65Q/QO/QP, ASTM A106, Gr. B, C, A53 Gr. B, A333 Gr. 6, ASME SA106 

In [3]:
nest_asyncio.apply()

async def buka_kelompok_barang(kd):
    """Membuka URL kelompok barang berdasarkan input kd."""
    driver = webdriver.Chrome()
    try:
        url = f"https://tkdn.kemenperin.go.id/sertifikat_idx.php?kd={kd}"
        driver.get(url)
        print(f"Berhasil membuka URL: {url}")
        return driver
    except Exception as e:
        print(f"Terjadi kesalahan: {e}")
        return None

async def scrap_data_kelompok_barang_pagination(driver, kd, page_limit=None):
    """Melakukan scrapping data dari tabel kelompok barang dengan pagination."""
    if driver is None:
        return

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div/div/div/div[1]/div[2]/table/tbody'))
        )

        header = ["perusahaan", "jenis_produk", "spesifikasi", "tipe", "merk", "nilai_tkdn"]
        print("\t".join(header))

        page_number = 1
        all_data = []

        while True:
            rows = driver.find_elements(By.XPATH, '//*[@id="content"]/div/div/div/div[1]/div[2]/table/tbody/tr')

            for row in rows:
                cells = row.find_elements(By.XPATH, './td[position()>=2 and position()<=7]')
                row_data = [cell.text.strip() for cell in cells]
                print("\t".join(row_data))
                all_data.append(dict(zip(header, row_data)))

            next_page = driver.find_elements(By.XPATH, '//*[@id="content"]/div/div/div/div/div[1]/div/div[2]/ul/li[position()=last()]/a')

            if next_page and "..." not in next_page[0].text:
                if page_limit and page_number >= page_limit:  # Check page limit
                    break
                next_page[0].click()
                page_number += 1
                time.sleep(2)
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div/div/div/div[1]/div[2]/table/tbody'))
                )
            else:
                break
        return all_data

    except Exception as e:
        print(f"Terjadi kesalahan saat scrapping data: {e}")
    finally:
        driver.quit()

def create_folder(kd):
    """Membuat folder hasil scrapping."""
    folder_name = f"hasil-scrapping-kd-{kd}"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    return folder_name

def save_data_to_json(data, kd):
    """Menyimpan data ke file JSON, dimulai dari id_data = 1, dan menambahkan kategori kelompok barang."""
    if not data:
        return

    kelompok_barang_dict = {
        1: "Bahan Penunjang Pertanian",
        2: "Mesin dan Peralatan Pertanian",
        3: "Mesin dan Peralatan Pertambangan",
        4: "Mesin dan Peralatan Migas",
        5: "Alat Berat, Konstruksi dan Material Handling",
        6: "Mesin dan Peralatan Pabrik",
        7: "Bahan Bangunan/Konstruksi",
        8: "Logam dan Barang Logam",
        9: "Bahan Kimia dan Barang Kimia",
        10: "Peralatan Elektronika",
        11: "Peralatan Kelistrikan",
        12: "Peralatan Telekomunikasi",
        13: "Alat Transport",
        14: "Bahan dan Peralatan Kesehatan",
        15: "Peralatan Laboratorium",
        16: "Komputer dan Peralatan Kantor",
        17: "Pakaian dan Perlengkapan Kerja",
        18: "Peralatan Olahraga dan Pendidikan",
        19: "Sarana Pertahanan",
        20: "Barang Lainnya",
    }
    
    kelompok_barang = kelompok_barang_dict.get(kd, "Kelompok Barang Tidak Diketahui")

    folder_name = create_folder(kd)
    now = datetime.now()
    timestamp = now.strftime("%Y-%m-%d--%H-%M-%S")
    file_name = f"hasil-scrapping-kd-{kd}-{timestamp}.json"
    file_path = os.path.join(folder_name, file_name)

    # Mengubah data ke dictionary dengan id_data dimulai dari 1
    json_data = []
    for i, row in enumerate(data[1:], start=1):  # Mulai dari index 1 (baris ke-1)
        # Menambahkan kelompok barang ke dictionary
        row_with_kelompok = {"Kelompok Barang": kelompok_barang, **row}
        json_data.append({"id_data": i, **row_with_kelompok})  # Gabungkan id_data dengan data lainnya

    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)
    print(f"Data disimpan di: {file_path}")

async def main():
    """Fungsi utama untuk menjalankan skrip asinkron."""
    kelompok_barang_dict = {
        1: "Bahan Penunjang Pertanian",
        2: "Mesin dan Peralatan Pertanian",
        3: "Mesin dan Peralatan Pertambangan",
        4: "Mesin dan Peralatan Migas",
        5: "Alat Berat, Konstruksi dan Material Handling",
        6: "Mesin dan Peralatan Pabrik",
        7: "Bahan Bangunan/Konstruksi",
        8: "Logam dan Barang Logam",
        9: "Bahan Kimia dan Barang Kimia",
        10: "Peralatan Elektronika",
        11: "Peralatan Kelistrikan",
        12: "Peralatan Telekomunikasi",
        13: "Alat Transport",
        14: "Bahan dan Peralatan Kesehatan",
        15: "Peralatan Laboratorium",
        16: "Komputer dan Peralatan Kantor",
        17: "Pakaian dan Perlengkapan Kerja",
        18: "Peralatan Olahraga dan Pendidikan",
        19: "Sarana Pertahanan",
        20: "Barang Lainnya",
    }

    print("Daftar Kode Kelompok Barang:")
    for kd, nama_kelompok in kelompok_barang_dict.items():
        print(f"{kd}: {nama_kelompok}")

    kd_input = int(input("Masukkan kode kelompok barang (1-20): "))  # Perubahan: 1-20
    page_limit = int(input("Masukkan batas halaman (opsional, tekan Enter untuk semua halaman): ") or 0)
    if 1 <= kd_input <= 20:  # Perubahan: 1-20
        driver = await buka_kelompok_barang(kd_input)
        if driver:
            all_data = await scrap_data_kelompok_barang_pagination(driver, kd_input, page_limit if page_limit > 0 else None)
            save_data_to_json(all_data, kd_input)
    else:
        print("Kode kelompok barang tidak valid. Masukkan angka antara 1 dan 20.")  # Perubahan: 1-20

if __name__ == "__main__":
    asyncio.run(main())

Daftar Kode Kelompok Barang:
1: Bahan Penunjang Pertanian
2: Mesin dan Peralatan Pertanian
3: Mesin dan Peralatan Pertambangan
4: Mesin dan Peralatan Migas
5: Alat Berat, Konstruksi dan Material Handling
6: Mesin dan Peralatan Pabrik
7: Bahan Bangunan/Konstruksi
8: Logam dan Barang Logam
9: Bahan Kimia dan Barang Kimia
10: Peralatan Elektronika
11: Peralatan Kelistrikan
12: Peralatan Telekomunikasi
13: Alat Transport
14: Bahan dan Peralatan Kesehatan
15: Peralatan Laboratorium
16: Komputer dan Peralatan Kantor
17: Pakaian dan Perlengkapan Kerja
18: Peralatan Olahraga dan Pendidikan
19: Sarana Pertahanan
20: Barang Lainnya
Berhasil membuka URL: https://tkdn.kemenperin.go.id/sertifikat_idx.php?kd=8
perusahaan	jenis_produk	spesifikasi	tipe	merk	nilai_tkdn

PT SARANA BANGUN PUSAKA	Pagar Pembatas/Pengaman Jalan (Guard Rail)	Beam : 4320mm, tebal 2,7mm ; Post: 1800mm, tebal : 6mm ; Block Post 350mm dan 400mm, tebal : 4,5mm	Tipe A	-	56.79%
PT SARANA BANGUN PUSAKA	Pagar Pembatas/Pengaman Jalan 